# Imports

In [1]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.extraccion.utils.Txt2Numbers import Txt2Numbers
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

import pandas as pd

## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "1")
         .config("spark.executor.memory", "12g")
         .config("spark.executor.cores", "2")
         .config("spark.driver.memory", "10g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "64g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .config("spark.sql.legacy.timeParserPolicy", "LEGACY")
         .config("spark.sql.legacy.parquet.int96RebaseModeInWrite", "CORRECTED")
         .getOrCreate())


sc = spark.sparkContext
sc.addFile('/home/jovyan/Work/ej/paquetes/nlppen/nlppen.zip')
sc.uiWebUrl

21/11/30 22:16:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://69abb479a5e1:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia y aplicar filtro del dataset

In [3]:

anno_inf = 2010
anno_sup = 2022

terminos = {
    'se ordena': [r'\bse ordena\b', r'\bse le ordena\b', r'\bse les ordena\b'],
    'se condena': [r'\bse condena\b', r'\bse le condena\b', r'\bse les condena\b'],
    'plan': [r'\bplan\b']
}

# seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales2', filtro=f'anno >= {anno_inf} AND anno < {anno_sup}')
seleccion = Seleccion(terminos, spark, parquet_path='./datasets/estructurales2/extracciones2', datasets_path='./datasets/estructurales3')
# seleccion.filtrar_sentencias(preprocess=solo_portanto, keepRowEmpty=True, partition_cols=['anno'], precargar=False)

estructurales = SentenciasEstructurales(seleccion)

seleccion.sdf.count()

507951

In [10]:
spark.read.parquet('../../src/datasets/complete').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ').count()

96773

In [ ]:
seleccion.sdf.columns

# Formar dataset de sentencias estructurales

In [4]:
from pyspark.sql.types import *
columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType()),
    'se ordena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeOrdena(columnas, True, True)

columnas = {
    'se condena PER' : ArrayType(StringType()),
    'se condena LOC' : ArrayType(StringType()),
    'se condena ORG' : ArrayType(StringType()),
    'se condena MISC' : ArrayType(StringType()),
    'se condena GPE' : ArrayType(StringType()),
    'se condena Ent Pub' : ArrayType(StringType())
}
estructurales.separarSeCondena(columnas, True, True)

columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
estructurales.extraerExtension(columnas, True)

columnas = {
        'plazosDefinidos' : ArrayType(TimestampType())
    }
estructurales.plazosDefinidos(columnas, True)

columnas = {
    'FechaSolicitud' : TimestampType(),
}

estructurales.extrarFechaRecibido(columnas, True)

columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

columnas = {
    'inst internacionales' : ArrayType(StringType())
}

estructurales.extraerInstrumentosInternacionales(columnas, True)

columnas = {
    'derechos Norm' : ArrayType(StringType()),
    'derechos GenXPat' : ArrayType(StringType())
}
estructurales.extraerDerechos(columnas, True)

columnas = {
    'derechos Acotados' : ArrayType(StringType()),
    'derechos General' : ArrayType(StringType()),
    'derechos Fundamental' : ArrayType(StringType()),
    'derechos Humano' : ArrayType(StringType())
}

estructurales.extraerDerechosSinNormalizar(columnas, True)

columnas = {
    'citasFechas' : MapType(StringType(), TimestampType())
}
estructurales.extrarCitaSentenciasFecha(columnas, True)

sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
columnas = {
    'id_sentencia' : StringType(),
    'num_doc_oficial' : StringType(),
    'expediente_oficial' : StringType()
}
estructurales.agregarIDSentencia(columnas, sentencias, True)


21/11/30 16:23:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_4786/4238448357.py", line 43, in <module>
    estructurales.extraerNumeroSentencia(columnas, True)
  File "/home/jovyan/Work/ej/paquetes/nlppen/nlppen/sentencias_estructurales.py", line 110, in extraerNumeroSentencia
    resultado = (self.seleccion.sdf.rdd
  File "/opt/conda/lib/python3.9/site-packages/pyspark/sql/dataframe.py", line 85, in rdd
    jrdd = self._jdf.javaToPython()
  File "/opt/conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1303, in __call__
    answer = self.gateway_client.send_command(command)
  File "/opt/conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/lib/python3.9/site-packages/py4j/java_gateway.py", line 1200, in send_command
    answer

TypeError: object of type 'NoneType' has no len()

In [5]:
from pyspark.sql.types import *
sentencias = pd.read_csv("listaSentencias.csv", sep=';', encoding='latin-1')
columnas = {
        'citasVotDate' : MapType(StringType(), TimestampType()),
        'citasIDVoto' : MapType(StringType(), StringType())
    }
estructurales.extrarCitaSentenciasFecha(columnas, sentencias, True)

21/11/30 16:32:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[archivo: string, txt: string, cleanText: string, num: bigint, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, tema: string, Recurrente_ents: array<string>, Cantón_ents: array<string>, Fecha_ents: array<string>, Constitución_ents: array<bigint>, Redactor_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, tema_prob: double, termino_lst: array<string>, expediente: string, Fechahora_ents: array<string>, CitaSentencias_ents: array<string>, Recurrido_ents: array<string>, Reglamento_ents: array<string>, Magistrado_ents: array<string>, _id: string, seguimiento: int, se_ordena: int, se_condena: int, plan: int, plazo: int, se_ordena_PER: array<string>, se_ordena_LOC: array<string>, se_ordena_ORG: array<string>, se_ordena_MISC: array<string>, se_ordena_GPE: array<string>, se_ordena_Ent_Pub: array<string>, se_condena_PER: array<string>, se_condena_LOC: array<string>, se_condena_ORG: arr

In [6]:
#Palabras de seeguimiento de acuerdo al correo de Eve.
columnas = {
        'seguimientoExt' : StringType(), #Texto extraido de seguimiento
        'seguimientoValue' : IntegerType() #Cantidad de ocurrencias
    }
estructurales.extraerSeguimiento(columnas, True)


DataFrame[archivo: string, txt: string, cleanText: string, num: bigint, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, tema: string, Recurrente_ents: array<string>, Cantón_ents: array<string>, Fecha_ents: array<string>, Constitución_ents: array<bigint>, Redactor_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, tema_prob: double, termino_lst: array<string>, expediente: string, Fechahora_ents: array<string>, CitaSentencias_ents: array<string>, Recurrido_ents: array<string>, Reglamento_ents: array<string>, Magistrado_ents: array<string>, _id: string, seguimiento: int, se_ordena: int, se_condena: int, plan: int, plazo: int, se_ordena_PER: array<string>, se_ordena_LOC: array<string>, se_ordena_ORG: array<string>, se_ordena_MISC: array<string>, se_ordena_GPE: array<string>, se_ordena_Ent_Pub: array<string>, se_condena_PER: array<string>, se_condena_LOC: array<string>, se_condena_ORG: arr

In [4]:
from pyspark.sql.types import *

#NOTA IMPORTANTE PARA KENNETH
#Plazos hay que correrlo de nuevo, las lines de abajo sobreescriben la columna 'plazosDefinidos' que se corrió previamente
columnas = {
        'plazosDefinidos' : ArrayType(TimestampType()) #Correción día 27-11-2021. Correr de nuevo
    }
estructurales.plazosDefinidos(columnas, actualizar_sdf=True, overwriteColumns=True)
    

21/11/30 15:16:44 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[archivo: string, txt: string, cleanText: string, num: bigint, fechahora_ext: timestamp, redactor_ext: string, tipoResolucion_ext: string, termino_ext: string, voto_salvado: boolean, tema: string, Recurrente_ents: array<string>, Cantón_ents: array<string>, Fecha_ents: array<string>, Constitución_ents: array<bigint>, Redactor_ents: array<string>, Ley_ents: array<string>, EntidadPublica_ents: array<string>, tema_prob: double, termino_lst: array<string>, expediente: string, Fechahora_ents: array<string>, CitaSentencias_ents: array<string>, Recurrido_ents: array<string>, Reglamento_ents: array<string>, Magistrado_ents: array<string>, _id: string, seguimiento: int, se_ordena: int, se_condena: int, plan: int, plazo: int, se_ordena_PER: array<string>, se_ordena_LOC: array<string>, se_ordena_ORG: array<string>, se_ordena_MISC: array<string>, se_ordena_GPE: array<string>, se_ordena_Ent_Pub: array<string>, se_condena_PER: array<string>, se_condena_LOC: array<string>, se_condena_ORG: arr

In [7]:
#Sobreescribir el dataset de filtro de sentencias con las nuevas columnas
estructurales.seleccion.guardarDatos(parquet_file='extracciones3', partition_cols=['anno'], borrar=False)

21/11/30 18:16:58 WARN MemoryStore: Not enough space to cache rdd_33_19 in memory! (computed 133.0 MiB so far)
21/11/30 18:16:58 WARN BlockManager: Persisting block rdd_33_19 to disk instead.
21/11/30 18:18:47 WARN MemoryStore: Not enough space to cache rdd_33_23 in memory! (computed 134.9 MiB so far)
21/11/30 18:18:47 WARN BlockManager: Persisting block rdd_33_23 to disk instead.
21/11/30 18:19:32 WARN MemoryStore: Not enough space to cache rdd_33_20 in memory! (computed 69.3 MiB so far)
21/11/30 18:19:32 WARN BlockManager: Persisting block rdd_33_20 to disk instead.
21/11/30 18:24:34 WARN MemoryStore: Not enough space to cache rdd_33_17 in memory! (computed 4.0 MiB so far)
21/11/30 18:24:34 WARN BlockManager: Persisting block rdd_33_17 to disk instead.
21/11/30 18:39:38 WARN MemoryStore: Not enough space to cache rdd_20_7 in memory! (computed 263.5 MiB so far)
21/11/30 18:39:38 WARN BlockManager: Persisting block rdd_20_7 to disk instead.
21/11/30 18:40:26 WARN MemoryStore: Not enoug

In [6]:
s = estructurales.seleccion.sdf

s.filter(s.plazosDefinidos.isNotNull()).count()
# s.select('plazosDefinidos').dropna().count()
# Cantidad anterior 30856
# Cantidad con el reprocesamiento 63446

21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_8 in memory! (computed 69.0 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_14 in memory! (computed 68.6 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_5 in memory! (computed 68.8 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_3 in memory! (computed 69.4 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_26 in memory! (computed 68.4 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_28 in memory! (computed 68.5 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_27 in memory! (computed 68.7 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_25 in memory! (computed 69.1 MiB so far)
21/11/30 15:37:53 WARN MemoryStore: Not enough space to cache rdd_20_29 in memory! (computed 68.3 MiB so far)
21/11/30 15:3

63446

In [6]:
estructurales.seleccion.sdf.limit(10).toPandas().columns

Index(['archivo', 'txt', 'cleanText', 'num', 'fechahora_ext', 'redactor_ext',
       'tipoResolucion_ext', 'termino_ext', 'voto_salvado', 'tema',
       'Recurrente_ents', 'Cantón_ents', 'Fecha_ents', 'Constitución_ents',
       'Redactor_ents', 'Ley_ents', 'EntidadPublica_ents', 'tema_prob',
       'termino_lst', 'expediente', 'Fechahora_ents', 'CitaSentencias_ents',
       'Recurrido_ents', 'Reglamento_ents', 'Magistrado_ents', '_id', 'anno',
       'seguimiento', 'se_ordena', 'se_condena', 'plan', 'plazo'],
      dtype='object')

In [4]:
columns = ['archivo',
 'num',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'tema',
 'Recurrente_ents',
 'Cantón_ents',
 'Fecha_ents',
 'Constitución_ents',
 'Redactor_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'tema_prob',
 'termino_lst',
 'expediente',
 'Fechahora_ents',
 'CitaSentencias_ents',
 'Recurrido_ents',
 'Reglamento_ents',
 'Magistrado_ents',
 '_id',
 'seguimiento',
 'se_ordena',
 'se_condena',
 'plan',
 'plazo',
 'se_ordena_PER',
 'se_ordena_LOC',
 'se_ordena_ORG',
 'se_ordena_MISC',
 'se_ordena_GPE',
 'se_ordena_Ent_Pub',
 'se_condena_PER',
 'se_condena_LOC',
 'se_condena_ORG',
 'se_condena_MISC',
 'se_condena_GPE',
 'se_condena_Ent_Pub',
 'extension_sentencia',
 'extension_por_lo_tanto',
 'plazosDefinidos',
 'FechaSolicitud',
 'num_resolucion',
 'inst_internacionales',
 'derechos_Norm',
 'derechos_GenXPat',
 'derechos_Acotados',
 'derechos_General',
 'derechos_Fundamental',
 'derechos_Humano',
 'citasFechas',
 'id_sentencia',
 'num_doc_oficial',
 'expediente_oficial',
 'citasVotDate',
 'citasIDVoto',
 'seguimientoExt',
 'seguimientoValue',
 'anno']

In [6]:
# temp = spark.read.parquet('./datasets/estructurales3/extracciones3').filter('termino_ext == "Con lugar" OR  termino_ext == "Con lugar parcial" ')
# temp = temp.filter('tema_prob > 0.90').filter('se_ordena != 0').sample(100/temp.count())

# temp.select(*columns).toPandas().to_csv('muestra de sentencias estructurales.csv')
temp.select(*columns).toPandas().to_parquet('muestra de sentencias estructurales.parquet')

/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:87: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.pyspark.enabled' is set to true; however, failed by the reason below:
  Unsupported type in conversion to Arrow: ArrayType(TimestampType,true)
Attempting non-optimization as 'spark.sql.execution.arrow.pyspark.fallback.enabled' is set to true.
  warnings.warn(msg)


# Cargar procesamiento de sentencias estructurales.

In [14]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b'],
    'plazo': [r'\bplazo\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='../../src/datasets/complete', datasets_path='./datasets/estructurales2/extracciones2')
seleccion.cargarPreprocesados()

estructurales = SentenciasEstructurales(seleccion)
seleccion.sdf.columns

['archivo',
 'txt',
 'cleanText',
 'num',
 'fechahora_ext',
 'redactor_ext',
 'tipoResolucion_ext',
 'termino_ext',
 'voto_salvado',
 'tema',
 'Recurrente_ents',
 'Cantón_ents',
 'Fecha_ents',
 'Constitución_ents',
 'Redactor_ents',
 'Ley_ents',
 'EntidadPublica_ents',
 'tema_prob',
 'termino_lst',
 'expediente',
 'Fechahora_ents',
 'CitaSentencias_ents',
 'Recurrido_ents',
 'Reglamento_ents',
 'Magistrado_ents',
 '_id',
 'anno']

In [ ]:
from pyspark.sql.types import *
columnas = {
    'num resolucion' : StringType()
}

estructurales.extraerNumeroSentencia(columnas, True)

# Zona de trabajo (ignorar)

In [ ]:
s = estructurales.seleccion.sdf
#print(solo_portanto(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'txt']))

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'se_ordena_PER'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1,'plazosDefinidos'])

In [ ]:
#print(s.where(s.plazosDefinidos.isNotNull()).limit(15).toPandas().loc[1])